In [1]:
import sys
import os
import sqlite3
import textwrap
import pandas as pd

In [2]:
from src.config import DB_PATH
sys.path.append(os.path.abspath('..'))
print(f"Database found at: {DB_PATH}")

Database found at: /home/hans/projects/journalist_dashboard/data/yle_data.db


In [3]:
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

In [4]:
cursor.execute("SELECT COUNT(*) FROM articles")
total_articles = cursor.fetchone()[0]

cursor.execute("SELECT COUNT(*) FROM articles WHERE content IS NOT NULL")
articles_with_content = cursor.fetchone()[0]

print(f"Total Articles: {total_articles}")
print(f"Articles with Content: {articles_with_content}")

Total Articles: 479
Articles with Content: 479


In [5]:
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, conn)

print(tables)

          name
0  journalists
1     articles


Let's check the structure of the 'articles' table

In [6]:
df_structure = pd.read_sql("PRAGMA table_info(articles);", conn)
print(df_structure[['name', 'type', 'pk']])

             name  type  pk
0              id  TEXT   1
1           title  TEXT   0
2             url  TEXT   0
3  published_date  TEXT   0
4         content  TEXT   0
5     description  TEXT   0
6        keywords  TEXT   0
7   journalist_id  TEXT   0


Check the structure of 'journalists' table

In [7]:
df_structure = pd.read_sql("PRAGMA table_info(journalists);", conn)
print(df_structure[['name', 'type', 'pk']])

          name  type  pk
0           id  TEXT   1
1         name  TEXT   0
2  profile_url  TEXT   0


Let's check some articles

In [11]:
df_articles_preview = pd.read_sql("SELECT * FROM articles;", conn)
print(df_articles_preview)

              id                                              title  \
0    74-20203401  Buollašat váikkuhit sakka maid dáluide – ná ba...   
1    74-20105252  Poikkeuksellinen ilmiö: Rotat valtasivat kylän...   
2    74-20105268  Jurssánis juŋkkát ráfehuhttet giliolbmuid Dean...   
3    74-20103823  Ohcejohka lea miestaluvvan šibihiid váilevuođa...   
4    74-20104142  Ohcejotgeainnu bajildanbargu lea farga gárvvis...   
..           ...                                                ...   
474    3-9672862  Kriittiset tekstiviestit ihmetyttivät Putinin ...   
475    3-9671475  Yle seurasi Putinin Suoraa linjaa – televisior...   
476    3-9658586  Venäläissivusto löysi todisteita: Venäjän soti...   
477    3-9655795  Näin venäläishakkerit nousivat julkisuuteen – ...   
478    3-9653659  Venäjä kiistää hakkeroinnin – Millainen toimij...   

                              url             published_date  \
0    https://yle.fi/a/74-20203401  2026-01-09T17:00:40+02:00   
1    https://yle.fi

In [10]:
df_journalists_preview = pd.read_sql("SELECT * FROM journalists;", conn)
print(df_journalists_preview)

           id               name                     profile_url
0  56-74-1533  Hans Ravna-Pieski  https://yle.fi/p/56-74-1533/fi
1   56-74-263   Vihtori Koskinen   https://yle.fi/p/56-74-263/fi
2  56-74-1051         Iida Tikka  https://yle.fi/p/56-74-1051/fi
3   56-74-138      Linnea Rasmus   https://yle.fi/p/56-74-138/fi
4  56-74-1937      Panu Pokkinen  https://yle.fi/p/56-74-1937/fi


In [34]:
cursor.execute("SELECT title, url, content FROM articles ORDER BY RANDOM() LIMIT 3")
rows = cursor.fetchall()

for i, row in enumerate(rows):
    title = row[0]
    url = row[1]
    content = row[2]
    
    print(f"Article #{i+1}")
    print(f"Title:   {title}")
    print(f"URL:     {url}")
    
    if content:
        preview = textwrap.shorten(content, width=500, placeholder="...")
        print(f"Content:\n{content}")
    else:
        print("Content: [EMPTY]")
        
    print("-" * 67)

Article #1
Title:   Sámemusea Siiddas sávvet nammadeami jagi eurohpalaš musean bidjat fártta sámedávviriid máhcaheapmái máilmmi museain
URL:     https://yle.fi/a/74-20087333
Content:
Sámemusea Siida Anáris lea nammaduvvon Jagi eurohpalaš musean 2024. Vuoiti bálkkašuvvui lávvardaga European Museum of the Year Award -konferánssas Portugala Portimãos. Bálkkašumis gilvaledje 50 musea 24 riikkas.

Bálkkašumi leigga Portugalis vuostáiváldimin museahoavda Taina Pieski ja intendeanta Eija Ojanlatva.

– Mii háliidit giitit olles Sámi servodaga dán bálkkašumi ovddas, dajai Pieski bálkkašumi oaččodettiin.

Bálkkašupmái nammaduvvojit ođđa ja ođasmahtton museat. Sámemusea Siiddas rahpe ođđa váldočájáhusa geassit 2022.

Bálkkašupmi mieđihuvvui searvvušlašvuođas ja kulturárbbi máhcaheapmái laktáseaddji barggus. Álbmotmusea máhcahii badjel 2 200 sámedávvira ruovttoluotta Sápmái sámemusea Siidii čakčat 2021.

– Mii válddiimet badjel 300 sápmelačča mielde bargui, mii laktásii min máttarváhnemiid dávviri

Everything as expected!

In [35]:
conn.close()

Print teh full structure of the database

In [ ]:
import pandas as pd
import sqlite3

# Connect to database
conn = sqlite3.connect('data/yle_data.db')

print("=== DATABASE SUMMARY ===\n")

# schema
print("--- TABLE SCHEMAS ---")
try:
    print("ARTICLES Columns:", pd.read_sql("SELECT * FROM articles LIMIT 0", conn).columns.tolist())
    print("JOURNALISTS Columns:", pd.read_sql("SELECT * FROM journalists LIMIT 0", conn).columns.tolist())
except Exception as e:
    print(f"Error reading schema: {e}")

# data sample
print("\n--- SAMPLE DATA ---")

try:
    df_journalist = pd.read_sql("SELECT id, name FROM journalists LIMIT 1 OFFSET 1", conn)
    
    if df_journalist.empty:
        print("something went wrong")
    else:
        target_id = df_journalist.iloc[0]['id']
        target_name = df_journalist.iloc[0]['name']
        print(f"Targeting Journalist: {target_name} (ID: {target_id})")

        # get first article
        sample_query = """
        SELECT 
            j.name AS journalist_name,
            a.journalist_id,
            a.title, 
            a.published_date,
            a.url,
            a.description,
            a.keywords,
            SUBSTR(a.content, 1, 200) || '...' as content_preview
        FROM articles a
        LEFT JOIN journalists j ON a.journalist_id = j.id
        WHERE a.journalist_id = ?
        LIMIT 1 OFFSET 2
        """
        df_sample = pd.read_sql(sample_query, conn, params=(target_id,))
        if not df_sample.empty:
            row = df_sample.iloc[0]
            for col_name, value in row.items():
                print(f"# {col_name.upper()}:\n   {value}\n")

except Exception as e:
    print(f"Error fetching sample: {e}")
    
print("\n--- ROW COUNTS ---")
try:
    print("Total Articles:", pd.read_sql("SELECT COUNT(*) FROM articles", conn).iloc[0,0])
    print("Total Journalists:", pd.read_sql("SELECT COUNT(*) FROM journalists", conn).iloc[0,0])
except:
    pass

conn.close()

=== DATABASE SUMMARY ===

--- TABLE SCHEMAS ---
ARTICLES Columns: ['id', 'title', 'url', 'published_date', 'content', 'description', 'keywords', 'journalist_id']
JOURNALISTS Columns: ['id', 'name', 'profile_url']

--- SAMPLE DATA ---
Targeting Journalist: Vihtori Koskinen (ID: 56-74-263)
# JOURNALIST_NAME:
   Vihtori Koskinen

# JOURNALIST_ID:
   56-74-263

# TITLE:
   Lahti kaatoi 15 katupuuta ja se ei kaikkia miellytä – ”Välillä se kommentointi on nykymaailman tapaan aika rankkaa”

# PUBLISHED_DATE:
   2025-12-09T16:00:58+02:00

# URL:
   https://yle.fi/a/74-20197668

# DESCRIPTION:
   Kaupunkipuita on poistettava, jos ne ovat liian lahoja ja paikalla, jossa huono­kuntoisista puista voi olla vaaraa ihmisille. Kaataminen saattaa herättää silti voimakkaita tunteita.

# KEYWORDS:
   ympäristö, kaupunkiluonto, puut, lehmukset, Lahti, Paavola, Lahti, Päijät-Häme

# CONTENT_PREVIEW:
   Lahden kaupunki on poistanut liudan katujen reunoilla kasvavia puita. 15 puuta kaadettiin keskiviikkona V